In [6]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup

In [56]:
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [24]:
url='https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.2178044&myY=126.8357822&searchKeyword='
driver.get(url)

In [25]:
#입력창에 광주를 입력하고 검색 버튼을 누르는 동작
driver.find_element(By.ID,'searchKeyword').send_keys('광주')
driver.find_element(By.ID,'fsSearchBtn').click()

In [26]:
### 이동 후에는 BeautifulSoup 이용하여 정보를 회득
soup = BeautifulSoup(driver.page_source,'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [27]:
li =lis[0]
name = li.select_one('.title').get_text().strip()
name

'광산소방서_광주-평동-119 안전센터'

In [29]:
addr = li.find('address').string.strip()
addr

'광주광역시 광산구 월전동 1251'

In [30]:
tel = li.select_one('.tel').get_text().strip()
tel

'전화공사중'

In [31]:
lines= []
for li in lis:
  name = li.select_one('.title').get_text().strip()
  addr = li.find('address').string.strip()
  tel = li.select_one('.tel').get_text().strip()
  lines.append([name,addr,tel])


In [32]:
driver.find_element(By.XPATH,'//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[2]/a').click()

In [33]:
soup = BeautifulSoup(driver.page_source,'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [ ]:
for li in lis:
  name = li.select_one('.title').get_text().strip()
  addr = li.find('address').string.strip()
  tel = li.select_one('.tel').get_text().strip()
  lines.append([name,addr,tel])

다음페이지

In [34]:
driver.find_element(By.CSS_SELECTOR,'.next_page').click()

전체

In [36]:
# 검색결과 건수 획득
strongs=soup.select('.stations-local-after > p > strong')
strongs[1].get_text()

'35건'

In [38]:
count = int(strongs[1].get_text().replace('건','').strip())
count

35

In [39]:
import math 
count = math.ceil(count/10)
count

4

최종코드

In [57]:
driver.get(url)
time.sleep(1)
driver.find_element(By.ID,'searchKeyword').send_keys('광주광역시')
driver.find_element(By.ID,'fsSearchBtn').click()

In [58]:
strongs=soup.select('.stations-local-after > p > strong')
count = int(strongs[1].get_text().replace('건','').strip())
count = math.ceil(count/10)

In [59]:
lines = []
for i in range(count):
  if i >= i and i %2==0:
    driver.find_element(By.CSS_SELECTOR,'.next_page').click()
    time.sleep(2)
  if i >= i and i %2==1:
    driver.find_element(By.XPATH,'//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[2]/a').click()
    time.sleep(2)
  soup = BeautifulSoup(driver.page_source,'html.parser')
  lis = soup.select('.stations-list > li')
  for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.find('address').string.strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append([name,addr,tel])

In [60]:
len(lines)

28

In [61]:
driver.close()

In [62]:
df = pd.DataFrame(lines, columns=['관서명','주소','전화번호'])
df.tail()

,관서명,주소,전화번호
23,광산소방서,광주광역시 광산구 하남산단1번로 13(하남동),062-613-8813
24,남부소방서,광주광역시 남구 송암로58번길 13(송하동),062-613-8613
25,동부소방서,광주광역시 동구 제봉로 210(대인동),062-613-8413
26,북부소방서,광주광역시 북구 서하로 290(오치동),062-613-8713
27,서부소방서,광주광역시 서구 화운로 197(화정동),062-613-8513
